In [1]:
import os
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, LSTM, Embedding, Dense
from tensorflow.keras.callbacks import ModelCheckpoint
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.utils import to_categorical

In [2]:
src_dataset = np.load(file='user_source.npy', allow_pickle=True)
tar_dataset = np.load(file='user_target.npy', allow_pickle=True)

print(len(src_dataset))
print(len(tar_dataset))

426
426


In [3]:
src_df = pd.DataFrame({'src':[]})
for i in range(len(src_dataset)) :
    src_df.loc[i] = [src_dataset[i]]

tar_df = pd.DataFrame({'tar':[]})
for i in range(len(tar_dataset)) :
    tar_df.loc[i] = [tar_dataset[i]]

df = pd.concat([src_df, tar_df], axis=1)
df.head()

,src,tar
0,"[[39.9844254, 116.3101288], [39.9841684, 116.2...","[[39.984702, 116.318417], [39.984683, 116.3184..."
1,"[[39.990832, 116.309828], [39.988258, 116.2933...","[[39.995777, 116.286798], [39.996832, 116.2854..."
2,"[[40.0087849, 116.3218099], [40.0089858, 116.3...","[[40.008921, 116.321484], [40.008582, 116.3222..."
3,"[[40.00815, 116.3204629], [40.0092487, 116.318...","[[40.009351, 116.321916], [40.009336, 116.3218..."
4,"[[40.00815, 116.3204629], [40.0088233, 116.321...","[[40.006747, 116.318549], [40.006747, 116.3185..."


In [ ]:
# <sos> '\t', <eos> '\n'
#df.tar = df.tar.apply(lambda x : '\t' + x + '\n')
#lines.sample(10)

In [ ]:
# coordinate set
src_coordinate = set()
for line in lines.src :
    for i in range(len(line)) :
        src_coordinate.add(line[i])

tar_coordinate = set()
for line in lines.tar :
    for i in range(len(line)) :
        tar_coordinate.add(line[i])

# size of coordinate set
src_coordinate_size = len(src_coordinate) + 1
tar_coordinate_size = len(tar_coordinate) + 1

# sort and assign index -> check error
#src_coordinate = sorted(list(src_coordinate))
#tar_coordinate = sorted(list(tar_coordinate))

# {[lat, lon]:1, [lat, lon]:2, ...}
src_to_index = dict([(coordinate, i+1) for i, coordinate in enumerate(src_coordinate)])
tar_to_index = dict([(coordinate, i+1) for i, coordinate in enumerate(tar_coordinate)])

In [ ]:
# encoding
encoder_input = []
for line in lines.src :
    temp_X = []
    for c in line :
        temp_X.append(src_to_index[c])
    encoder_input.append(temp_X)
#print(encoder_input[:5])

In [ ]:
# decoding
decoder_input = []
for line in lines.tar :
    temp_X = []
    for c in line :
        temp_X.append(tar_to_index[c])
    decoder_input.append(temp_X)
#print(decoder_input[:5])

In [ ]:
# !!! this code might be not used !!!
# remove start symbol of target which is used for test
decoder_target = []
for line in lines.tar :
    t = 0
    temp_X = []
    for c in line :
        if t > 0 :
            temp_X.append(tar_to_index[c])
        t += 1
    decoder_target.append(temp_X)
#print(decoder_target[:5])

In [ ]:
# padding
max_src_len = max([len(line) for line in lines.src])
max_tar_len = max([len(line) for line in lines.tar])
#print(max_src_len)
#print(max_tar_len)

encoder_input = pad_sequences(encoder_input, maxlen=max_src_len, padding='post')
decoder_input = pad_sequences(decoder_input, maxlen=max_tar_len, padding='post')
decoder_target = pad_sequences(decoder_target, maxlen=max_tar_len, padding='post')

In [ ]:
# one-hot encoding
encoder_input = to_categorical(encoder_input)
decoder_input = to_categorical(decoder_input)
decoder_target = to_categorical(decoder_target)

In [ ]:
# seq2seq model teacher forcing
encoder_inputs = Input(shape=(None, src_coordinate_size))
# hidden state size is 256, return encoder state to decoder
encoder_lstm = LSTM(units=256, return_state=True)
# encoder outputs, hidden state, cell state
encoder_outputs, state_h, state_c = encoder_lstm(encoder_inputs)
encoder_states = [state_h, state_c]

decoder_inputs = Input(shape=(None, tar_coordinate_size))
decoder_lstm = LSTM(units=256, return_sequences=True, return_state=True)
# use last encoder state as initial state
decoder_outputs, _, _ = decoder_lstm(decoder_inputs, initial_state=encoder_states)
# assign nodes as size of target coordinate
decoder_softmax_layer = Dense(tar_coordinate_size, activation='softmax')
decoder_outputs = decoder_softmax_layer(decoder_outputs)

model = Model([encoder_inputs, decoder_inputs], decoder_outputs)
model.compile(optimizer='rmsprop', loss='categorical_crossentropy')

MODEL_DIR = './model/'
if not os.path.exists(MODEL_DIR) :
    os.mkdir(MODEL_DIR)

modelpath = './model/{epoch:02d}-{val_loss:.4f}.hdf5'
checkpointer = ModelCheckpoint(filepath=modelpath, monitor='val_loss', verbose=1, save_best_only=True)

In [ ]:
# !!! control batch_size, epochs to avoid overfitting !!!
model.fit(x=[encoder_input, decoder_input], y=decoder_target,
          batch_size=64, epochs=40, validation_split=0.2, callbacks=[checkpointer])

In [ ]:
#model.save('s2s.h5')